In [1]:
import pandas as pd
patients_df = pd.read_csv("/content/drive/MyDrive/dataset/Patient_Data.csv")
billing_df = pd.read_csv("/content/drive/MyDrive/dataset/Billing_Data.csv")

In [2]:
# 1. Show summary of patient dataset
print("Patient Dataset Info:")
print(patients_df.info())

Patient Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes
None


In [3]:
# 2. Select only relevant columns for billing
billing_relevant = patients_df[['PatientID', 'Department', 'Doctor', 'BillAmount']]

In [4]:
# 3. Drop administrative columns
patients_df = patients_df.drop(['ReceptionistID', 'CheckInTime'], axis=1, errors='ignore')


In [5]:
# 4. Groupby department to find total bill amount
dept_billing = billing_relevant.groupby('Department')['BillAmount'].sum().reset_index()
print("\nTotal Bill Amount per Department:")
print(dept_billing)


Total Bill Amount per Department:
    Department  BillAmount
0   Cardiology     16200.0
1  Dermatology         0.0
2    Neurology         0.0
3  Orthopedics      7500.0


In [6]:
# 5. Remove duplicate patient records
patients_df = patients_df.drop_duplicates(subset=['PatientID'])

In [7]:
# 6. Fill missing BillAmount values with mean
mean_bill = billing_relevant['BillAmount'].mean()
billing_relevant['BillAmount'] = billing_relevant['BillAmount'].fillna(mean_bill)


In [8]:
# 7. Merge billing dataset with patient dataset on PatientID
merged_df = pd.merge(patients_df, billing_df, on='PatientID', how='left')

In [9]:
# 8. Concatenate new patients (row-wise)
new_patients = pd.DataFrame({
    'PatientID': [501, 502],
    'Department': ['Cardiology', 'Neurology'],
    'Doctor': ['Dr. A', 'Dr. B'],
    'BillAmount': [2000, 3500]
})
patients_updated = pd.concat([patients_df, new_patients], ignore_index=True)

In [10]:
# 9. Concatenate new billing columns (column-wise)
new_billing_cols = pd.DataFrame({
    'InsuranceCovered': [True, False, True, False, True][:len(merged_df)],
    'FinalAmount': merged_df['BillAmount'].fillna(mean_bill) * 0.9  # Example 10% discount
})
final_df = pd.concat([merged_df, new_billing_cols], axis=1)

In [11]:
print("\nFinal Cleaned Dataset:")
print(final_df.head())


Final Cleaned Dataset:
   PatientID     Name   Department     Doctor  BillAmount  InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith      5000.0              2000   
1        102      Bob    Neurology   Dr. John         NaN              1500   
2        103  Charlie  Orthopedics    Dr. Lee      7500.0              2500   
3        104    David   Cardiology  Dr. Smith      6200.0              3000   
4        105      Eva  Dermatology   Dr. Rose         NaN              1000   

   FinalAmount  InsuranceCovered  FinalAmount  
0         3000              True       4500.0  
1         3500             False       5332.5  
2         5000              True       6750.0  
3         3200             False       5580.0  
4         4000              True       5332.5  
